### 完整的模型训练套路

In [12]:
import torch
import torchvision
from torch import nn
from torch.nn import *
from torch.utils.data import DataLoader

In [13]:
# 统一转换为 tensor 类型
dataset_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [14]:
# 训练集
train_set = torchvision.datasets.CIFAR10(root='./dataset/CIFAR10',train=True, transform=dataset_transform, download=True)
# 测试集
test_set = torchvision.datasets.CIFAR10(root='./dataset/CIFAR10',train=False, transform=dataset_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
train_set_size = len(train_set)
test_set_size = len(test_set)

In [16]:
# 创建加载数据的对象
train_loader = DataLoader(dataset=train_set, batch_size=64)
test_loader = DataLoader(dataset=test_set, batch_size=64)

In [17]:
class MyModel(nn.Module):
    # 重写构造函数，确定神经网络的构成
    def __init__(self):
        # 调用父类进行初始化
        super(MyModel, self).__init__()
        self.model1 = Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )
    # 重写前向运算
    def forward(self, x):
       return self.model1(x)

In [18]:
mymodel = MyModel()
loss = CrossEntropyLoss()
lr = 0.01
optim = torch.optim.SGD(mymodel.parameters(), lr=lr)

In [23]:
total_train_step = 0
for epoch in range(2):
    print("-----第 {} 轮训练-----".format(epoch+1))

    # 训练步骤
    mymodel.train()
    for data in train_loader:

        imgs, targets = data
        output = mymodel(imgs)

        # 求解损失函数
        result_loss = loss(output,targets)
        # 梯度初始化为0，不受之前求解的梯度影响
        optim.zero_grad()
        # 进行反向传播优化(求解梯度)
        result_loss.backward()
        # 利用梯度优化参数
        optim.step()

        total_train_step += 1
        # print("训练次数: {}, Loss: {}".format(total_train_step,result_loss.item()))
    
    # 测试步骤
    mymodel.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_loader:

            imgs, targets = data
            output = mymodel(imgs)

            # 求解损失函数
            result_loss = loss(output,targets)
            total_test_loss += result_loss.item()

            # 求解准确率
            accuracy = (output.argmax(1) == targets).sum()
            total_accuracy += accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的准确率: {}".format(total_accuracy/test_set_size))
    # 保存模型
    torch.save(mymodel,"model/CIFA10_{}.pth".format(epoch+1))
    print("已保存模型")


-----第 1 轮训练-----
整体测试集上的Loss: 245.44336187839508
整体测试集上的准确率: 0.43459999561309814
已保存模型
-----第 2 轮训练-----
整体测试集上的Loss: 234.46131908893585
整体测试集上的准确率: 0.4580000042915344
已保存模型
